In [1]:
import torch
import torch.nn as nn
import torch.optim
from torch.autograd import Variable
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
%matplotlib inline

In [2]:
samples = 2000
sz = 10
probability = 1.0 * np.array([10,6,4,3,1,1,1,1,1,1])
probability = probability[ : sz]
probability = probability/sum(probability)

In [3]:
train_set = []

for m in range(samples):
    n = np.random.choice(range(1,sz+1),p = probability)
    inputs =[0]* n + [1]*n
    inputs.insert(0,3)
    inputs.append(2)
    train_set.append(inputs)

In [4]:
valid_set = []

for m in range(samples//10):
    n = np.random.choice(range(1,sz+1),p = probability)
    inputs =[0]* n + [1]*n
    inputs.insert(0,3)
    inputs.append(2)
    valid_set.append(inputs)

In [5]:
for m in range(2):
    n = sz+m
    inputs =[0]* n + [1]*n
    inputs.insert(0,3)
    inputs.append(2)
    valid_set.append(inputs)

np.random.shuffle(valid_set)

In [6]:
class SimpleRNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,num_layers=1):
        super(SimpleRNN,self).__init__()
        self.hidden_size = hidden_size 
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size,hidden_size)
        self.rnn = nn.RNN(hidden_size,hidden_size,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size)
        self.softmax = nn.LogSoftmax(dim = 1)
    
    def forward(self,input,hidden):
        x = self.embedding(input)
        output,hidden =self.rnn(x,hidden)
        output= output[:,-1,:]
        output =self.fc(output)
        output= self.softmax(output)
        return output,hidden
    
    def initHidden(self):
        hidden = Variable(torch.zeros(self.num_layers,1,self.hidden_size))
        cell = Variable(torch.zeros(self.num_layers,1,self.hidden_size))
        return (hidden,cell)
    

In [7]:
rnn = SimpleRNN(input_size=4,hidden_size=2,output_size=3)
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(rnn.parameters(),lr = 0.001)

In [8]:
train_loss = 0

def trainRNN(epoch):
    global train_loss
    train_loss =0
    np.random.shuffle(train_set)

    for i,seq in enumerate(train_set):
        loss = 0
        hidden = rnn.initHidden()
        for t in range(len(seq)-1):
            x=Variable(torch.LongTensor([seq[t]]).unsqueeze(0))
            y =Variable(torch.LongTensor([seq[t+1]]))
            hidden = hidden[0] if isinstance(hidden, tuple) else hidden
            output, hidden = rnn(x, hidden)
            loss+=criterion(output,y)
            
        loss=1.0*loss/len(seq)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss+=loss
        if 1>0 and i%500 ==0:
            print('第{}轮，第{}个,训练Loss:{:.2f}'.format(epoch,
                                                   i,
                                                   train_loss.data.numpy()/(i+1)
                                                   ))

In [9]:
valid_loss = 0
errors = 0
show_out = ''

def evaluateRNN():
    global valid_loss
    global errors
    global show_out
    valid_loss = 0
    errors = 0
    show_out = ''
    for i ,seq in enumerate(valid_set):
        loss =0
        outstring = ''
        targets = ''
        diff = 0
        hidden = rnn.initHidden()
        for t in range(len(seq)-1):
            x = Variable(torch.LongTensor([seq[t]]).unsqueeze(0))
            y =Variable(torch.LongTensor([seq[t+1]]))
            hidden = hidden[0] if isinstance(hidden, tuple) else hidden
            output,hidden=rnn(x,hidden)
            mm = torch.max(output,1)[1][0]
            outstring +=str(mm.data.numpy())
            targets+=str(y.data.numpy()[0])
            loss+=criterion(output,y)
            diff +=1-mm.eq(y).data.numpy()[0]
        loss = 1.0*loss/len(seq)
        valid_loss+=loss
        errors+=diff
        if np.random.rand()<0.1:
            show_out= outstring+'\n'+targets
    print(output[0][2].data.numpy())




In [10]:
num_epoch = 20
results = []
for epoch in range(num_epoch):
    trainRNN(epoch)
    evaluateRNN()
    print('第{}轮,训练Loss:{:.2f},错误率:{:.2f}'.format(epoch,
                                                 train_loss.data.numpy()/len(train_set),
                                                 valid_loss.data.numpy()/len(valid_set),
                                                 1.0*errors/len(valid_set)
                                                 ))
    print(show_out)
    results.append([train_loss.data.numpy()/len(train_set),
                    valid_loss.data.numpy()/len(train_set),
                    1.0*errors/len(valid_set)
                    ])

第0轮，第0个,训练Loss:1.05
第0轮，第500个,训练Loss:0.81
第0轮，第1000个,训练Loss:0.72
第0轮，第1500个,训练Loss:0.66
-0.87028253
第0轮,训练Loss:0.63,错误率:0.53
000000000002111111111
000000000011111111112
第1轮，第0个,训练Loss:0.53
第1轮，第500个,训练Loss:0.52
第1轮，第1000个,训练Loss:0.51
第1轮，第1500个,训练Loss:0.51
-0.864993
第1轮,训练Loss:0.51,错误率:0.49
00000021111
00000111112
第2轮，第0个,训练Loss:0.54
第2轮，第500个,训练Loss:0.49
第2轮，第1000个,训练Loss:0.49
第2轮，第1500个,训练Loss:0.48
-0.8285055
第2轮,训练Loss:0.48,错误率:0.48
010000001111111
000000011111112
第3轮，第0个,训练Loss:0.54
第3轮，第500个,训练Loss:0.48
第3轮，第1000个,训练Loss:0.48
第3轮，第1500个,训练Loss:0.48
-0.83071256
第3轮,训练Loss:0.48,错误率:0.47
0000211
0001112
第4轮，第0个,训练Loss:0.54
第4轮，第500个,训练Loss:0.48
第4轮，第1000个,训练Loss:0.48
第4轮，第1500个,训练Loss:0.47
-0.7998311
第4轮,训练Loss:0.47,错误率:0.47
01000021111
00000111112
第5轮，第0个,训练Loss:0.53
第5轮，第500个,训练Loss:0.47
第5轮，第1000个,训练Loss:0.47
第5轮，第1500个,训练Loss:0.47
-0.8371579
第5轮,训练Loss:0.47,错误率:0.47
0100111
0001112
第6轮，第0个,训练Loss:0.53
第6轮，第500个,训练Loss:0.48
第6轮，第1000个,训练Loss:0.47
第6轮，第1500个,训练Loss:0.47
-0.8369648


In [11]:
torch.save(rnn,'rnn.mdl')
rnn=torch.load('rnn.mdl')

In [12]:
for n in range(20):
    inputs = [0]*n+[1]*n
    inputs.insert(0,3)
    inputs.append(2)
    outstring = ''
    targets = ''
    diff =0
    hiddens = []
    hidden = rnn.initHidden()
    for t in range(len(inputs)-1):
        x = Variable(torch.LongTensor([inputs[t]]).unsqueeze(0))
        y =Variable(torch.LongTensor([inputs[t+1]]))
        hidden = hidden[0] if isinstance(hidden, tuple) else hidden
        output,hidden=rnn(x,hidden)
        mm = torch.max(output,1)[1][0]
        outstring +=str(mm.data.numpy())
        targets+=str(y.data.numpy()[0])
        
        diff +=1-mm.eq(y).data.numpy()[0]
    print(n)
    print(outstring)
    print(targets)
    print('Diff:{}'.format(diff))

0
0
2
Diff:1
1
011
012
Diff:1
2
01021
00112
Diff:4
3
0100211
0001112
Diff:4
4
010002111
000011112
Diff:4
5
01000021111
00000111112
Diff:4
6
0100000211111
0000001111112
Diff:4
7
010000002111111
000000011111112
Diff:4
8
01000000021111111
00000000111111112
Diff:4
9
0100000000211111111
0000000001111111112
Diff:4
10
010000000002111111111
000000000011111111112
Diff:4
11
01000000000021111111111
00000000000111111111112
Diff:4
12
0100000000000211111111111
0000000000001111111111112
Diff:4
13
010000000000002111111111111
000000000000011111111111112
Diff:4
14
01000000000000021111111111111
00000000000000111111111111112
Diff:4
15
0100000000000000211111111111111
0000000000000001111111111111112
Diff:4
16
010000000000000002111111111111111
000000000000000011111111111111112
Diff:4
17
01000000000000000021111111111111111
00000000000000000111111111111111112
Diff:4
18
0100000000000000000211111111111111111
0000000000000000001111111111111111112
Diff:4
19
010000000000000000002111111111111111111
00000000000000000